In [ ]:
%load_ext autoreload
%autoreload 2

# Example Use of USB stream and Sockets
> Example usage  see link  
https://www.pythonforthelab.com/blog/using-pyzmq-for-inter-process-communication-part-1/


## Send and receive socket wih FLIR cam

In [ ]:

from time import sleep
from imutils.video import FPS
import imutils
import cv2
from FLIRCam.USB_video_stream import *

import zmq

print('Starting1')
context = zmq.Context()
pubsocket = context.socket(zmq.PUB)
pubsocket.bind("tcp://*:5555")

subsocket = context.socket(zmq.SUB)
subsocket.connect("tcp://localhost:5555")
subsocket.setsockopt(zmq.SUBSCRIBE, b'camera_frame')

width = 1000
height = 750

fvs = USBVideoStream() # .start()
fvs.start()

sleep(0.5)

topic = 'camera_frame'
i=0
while True:
    try:
        i += 1
        while not fvs.more():
            pass
        snd_frame = fvs.read()        
        snd_frame = imutils.resize(snd_frame['image'], width=width, height=height)   
        
        pubsocket.send_string(topic, zmq.SNDMORE)
        pubsocket.send_pyobj(snd_frame)

        rec_topic = subsocket.recv_string()
        rec_frame = subsocket.recv_pyobj()
        cv2.putText(rec_frame, f'Received frame number {i}, {rec_topic}, {rec_frame.shape}',
        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        cv2.imshow("Cam1", rec_frame)
        
        if cv2.waitKey(100) == 27: 
            break  # esc to quit
    
    except KeyboardInterrupt:
        print("W: interrupt received, stopping…")
        break
        
fvs.stop()  
del fvs

subsocket.close() 
pubsocket.close()  # clean up
cv2.destroyAllWindows()

print('Finished')

Starting1
Library version: 1.27.0.48
Number of cameras detected: 2
Camera acquisition mode set to continuous...
Camera 0 started acquiring images...

Camera acquisition mode set to continuous...
Camera 1 started acquiring images...

Finished


## Send and receive socket wih Webcam

In [ ]:
from time import sleep
import zmq
import cv2
print('Starting1')
context = zmq.Context()
pubsocket = context.socket(zmq.PUB)
pubsocket.bind("tcp://*:5555")

subsocket = context.socket(zmq.SUB)
subsocket.connect("tcp://localhost:5555")
subsocket.setsockopt(zmq.SUBSCRIBE, b'camera_frame')


print('Starting2')
cam = cv2.VideoCapture(0)
sleep(0.5)

topic = 'camera_frame'
print('Starting3')
i=0
while True:
    try:
        i += 1
        ret, snd_frame = cam.read()
        pubsocket.send_string(topic, zmq.SNDMORE)
        pubsocket.send_pyobj(snd_frame)

        rec_topic = subsocket.recv_string()
        rec_frame = subsocket.recv_pyobj()
        cv2.putText(rec_frame, f'Received frame number {i}, {rec_topic}, {rec_frame.shape}',
        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        cv2.imshow("Cam1", rec_frame)
        
        if cv2.waitKey(100) == 27: 
            break  # esc to quit
    
    except KeyboardInterrupt:
        print("W: interrupt received, stopping…")
        break
        
        
cam.release()
subsocket.close() 
pubsocket.close()  # clean up
cv2.destroyAllWindows()
print('Finished')

## Show FLIR camera

In [ ]:
def show_FLIRcam(mirror=False, width=1000, height=750):
    fvs = USBVideoStream().start()
    while True:
        while not fvs.more():
            pass
        img = fvs.read()        
        img = imutils.resize(img['image'], width=width, height=height)     
        if mirror: 
            img = cv2.flip(img, 1)
        cv2.imshow('FLIR cam', img)
        if cv2.waitKey(1) == 27: 
            break  # esc to quit
            
    fvs.stop()  
    del fvs
    cv2.destroyAllWindows()

show_FLIRcam(mirror=True)


Library version: 1.27.0.48
Number of cameras detected: 2
Camera acquisition mode set to continuous...
Camera 0 started acquiring images...

Camera acquisition mode set to continuous...
Camera 1 started acquiring images...



## Show OpenCV web camera

In [ ]:
def show_webcam(mirror=False):
    cam = cv2.VideoCapture(0)
    while True:
        ret_val, img = cam.read()
        if mirror: 
            img = cv2.flip(img, 1)
        cv2.imshow('my webcam', img)
        if cv2.waitKey(1) == 27: 
            break  # esc to quit
    cv2.destroyAllWindows()
    cam.release()

show_webcam(mirror=True)

